In [3]:
import pandas as pd

# data loading / analysis

In [4]:
# load 'Alternative EV Screen.xlsx'  
alt_ev_screen_df = pd.read_excel('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/Alternative EV Screen.xlsx', sheet_name='Cleaned')


print(len(alt_ev_screen_df))
print(alt_ev_screen_df.columns) 
display(alt_ev_screen_df.head())

52
Index(['Ticker', 'Last Price', 'YTD Returns', 'Cover Days', 'SI', 'NT Uptrend',
       'Intermediate Term Uptrend', '20 DMA', '50 DMA', '100 DMA', '200 DMA',
       '2Q24', '3Q24', '4Q24', 'ADV $M', 'Cash Runway (Qs)', 'Market Cap',
       'Total Debt', 'Current', '2Q24 EV', '3Q24 EV', 'Cash on Hand',
       'Net Loss (Qs)', '20 DMA.1', '50 DMA.1', '100 DMA.1', '200 DMA.1'],
      dtype='object')


/Users/sudz4/Desktop/SPS_local/sps/sps_venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/sudz4/Desktop/SPS_local/sps/sps_venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Ticker,Last Price,YTD Returns,Cover Days,SI,NT Uptrend,Intermediate Term Uptrend,20 DMA,50 DMA,100 DMA,...,Total Debt,Current,2Q24 EV,3Q24 EV,Cash on Hand,Net Loss (Qs),20 DMA.1,50 DMA.1,100 DMA.1,200 DMA.1
0,PRE,4.7700,-0.192893,1.056601,0.003922,-0.010658,-0.074336,0.035944,0.024903,-0.089601,...,2.369388,-124.177628,-112.677628,-101.177628,183.873483,-11.500,4.604495,4.654098,5.239461,5.027851
1,ANTX,1.1000,-0.946315,1.365465,0.006300,-0.007799,-0.729220,0.041667,0.033543,-0.334483,...,0.000000,-71.713326,-60.213326,-48.713326,104.540000,-11.500,1.056,1.064300,1.652850,3.930500
2,BOLT,0.6596,-0.411039,1.523604,0.003992,-0.000992,-0.282287,0.019971,0.018959,-0.044480,...,20.219000,-56.729132,-43.277132,-29.825132,102.189000,-13.452,0.646685,0.647327,0.690304,0.901930
3,ALGS,8.7500,-0.472892,1.754932,0.033707,-0.262578,-0.327272,0.128086,-0.168124,-0.230097,...,11.138000,-55.432097,-46.087097,-36.742097,94.536000,-9.345,7.7565,10.518400,11.365075,15.635444
4,KZR,0.7651,-0.192421,0.671468,0.017282,0.191504,-0.093192,-0.061710,0.117976,0.164678,...,18.933000,-89.425605,-66.425605,-43.425605,164.182000,-23.000,0.81542,0.684362,0.656920,0.754693


# data mining / validation

In [55]:
import requests
import pandas as pd
from datetime import datetime
import time

In [56]:
def get_company_info(cik, headers):
    """
    Get company SIC code and industry info from SEC
    """
    try:
        cik_padded = str(cik).zfill(10)
        url = f'https://data.sec.gov/submissions/CIK{cik_padded}.json'
        
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            return {
                'sic': data.get('sicCode', ''),
                'industry': data.get('sicDescription', ''),
                'ticker': data.get('tickers', [''])[0] if data.get('tickers') else ''
            }
    except Exception as e:
        print(f"Error fetching company info for CIK {cik}: {str(e)}")
    return None

def get_delistings(start_year=2020, save_csv=True):
    """
    Get Form 25 delistings using SEC's HTTPS access
    Returns: DataFrame of delistings
    Optional: Saves CSV file if save_csv=True
    """
    headers = {
        'User-Agent': 'SPS matt@sanspeursystems.com',
        'Accept-Encoding': 'gzip, deflate',
        'Host': 'www.sec.gov'
    }

    all_delistings = []
    current_year = datetime.now().year

    try:
        for year in range(start_year, current_year + 1):
            for quarter in range(1, 5):
                url = f'https://www.sec.gov/Archives/edgar/full-index/{year}/QTR{quarter}/master.idx'
                
                try:
                    print(f"Fetching {year} Q{quarter}...")
                    response = requests.get(url, headers=headers)
                    
                    if response.status_code == 200:
                        lines = response.text.split('\n')
                        data_started = False
                        
                        for line in lines:
                            if data_started:
                                if '|25|' in line:
                                    parts = line.split('|')
                                    if len(parts) >= 4:
                                        cik = parts[0]
                                        # company_info = get_company_info(cik, headers)
                                        
                                        delisting = {
                                            'date': parts[3],
                                            'company': parts[1],
                                            'cik': cik,
                                            'form': '25'
                                            # 'ticker': company_info['ticker'] if company_info else '',
                                            # 'industry': company_info['industry'] if company_info else '',
                                            # 'sic': company_info['sic'] if company_info else ''
                                        }
                                        all_delistings.append(delisting)
                            elif line.startswith('---'):
                                data_started = True
                                
                    time.sleep(0.1)  # SEC rate limiting
                    
                except requests.exceptions.RequestException as e:
                    print(f"Error fetching {year} Q{quarter}: {str(e)}")
                    continue

        if all_delistings:
            df = pd.DataFrame(all_delistings)
            df['date'] = pd.to_datetime(df['date'])
            df = df.sort_values('date', ascending=False)
            
            # Save CSV if requested
            if save_csv:
                csv_filename = f'all_delistings_since_{start_year}.csv'
                df.to_csv(csv_filename, index=False)
                print(f"\nSaved to {csv_filename}")
            
            return df
        else:
            print("No delistings found")
            return None

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# # Example usage
# if __name__ == "__main__":
#     print("Fetching delistings since 2020...")
    
#     # Get the DataFrame
#     df = get_delistings(2020, save_csv=True)
    
#     if df is not None and not df.empty:
#         print("\nRecent delistings summary:")
#         print(df[['date', 'ticker', 'company', 'industry']].head())
#         print(f"\nTotal delistings found: {len(df)}")

In [37]:
# execute 
all_delistings_df = get_delistings(2016, save_csv=False)

Fetching 2016 Q1...
Fetching 2016 Q2...
Fetching 2016 Q3...
Fetching 2016 Q4...
Fetching 2017 Q1...
Fetching 2017 Q2...
Fetching 2017 Q3...
Fetching 2017 Q4...
Fetching 2018 Q1...
Fetching 2018 Q2...
Fetching 2018 Q3...
Fetching 2018 Q4...
Fetching 2019 Q1...
Fetching 2019 Q2...
Fetching 2019 Q3...
Fetching 2019 Q4...
Fetching 2020 Q1...
Fetching 2020 Q2...
Fetching 2020 Q3...
Fetching 2020 Q4...
Fetching 2021 Q1...
Fetching 2021 Q2...
Fetching 2021 Q3...
Fetching 2021 Q4...
Fetching 2022 Q1...
Fetching 2022 Q2...
Fetching 2022 Q3...
Fetching 2022 Q4...
Fetching 2023 Q1...
Fetching 2023 Q2...
Fetching 2023 Q3...
Fetching 2023 Q4...
Fetching 2024 Q1...
Fetching 2024 Q2...
Fetching 2024 Q3...
Fetching 2024 Q4...


In [57]:
print(len(all_delistings_df))
display(all_delistings_df.head())

983


,date,company,cik,form
979,2024-10-31,"Finch Therapeutics Group, Inc.",1733257,25
982,2024-10-21,TELLURIAN INC. /DE/,61398,25
978,2024-10-11,"Guardion Health Sciences, Inc.",1642375,25
977,2024-10-11,Investment Managers Series Trust II,1587982,25
980,2024-10-07,Renalytix plc,1811115,25


SICK CoDE time

In [58]:
sick_df = all_delistings_df.copy()

In [59]:
# make sure CIK is 10 digits
sick_df['cik'] = sick_df['cik'].astype(str).str.zfill(10)
# convert date string to datetime object
sick_df['date'] = pd.to_datetime(sick_df['date'])

# add columns for data extraction preparation
sick_df['ticker'] = ''
sick_df['sic'] = ''
sick_df['sic_description'] = ''
sick_df['is_biotech'] = False
sick_df['biotech_identification_method'] = ''

In [60]:
print(len(sick_df))
display(sick_df.head())

983


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
979,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,,,,False,
982,2024-10-21,TELLURIAN INC. /DE/,0000061398,25,,,,False,
978,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,,,,False,
977,2024-10-11,Investment Managers Series Trust II,0001587982,25,,,,False,
980,2024-10-07,Renalytix plc,0001811115,25,,,,False,


In [61]:
# Define biotech-related keywords and patterns (expanded)
biotech_keywords = [
    'therapeutics', 'pharma', 'biotech', 'bio', 'health sciences',
    'medical', 'genetics', 'genomics', 'life sciences', 'biologics',
    'biosciences', 'oncology', 'diagnostic', 'biomedical',
    'laboratory', 'clinical', 'research', 'drug', 'therapeutic',
    'cell therapy', 'gene therapy', 'molecular', 'pharmaceutical'
]

# Define comprehensive biotech and related SIC codes with descriptions
biotech_sic_dict = {
    '2833': 'Medicinal Chemicals and Botanical Products',
    '2834': 'Pharmaceutical Preparations',
    '2835': 'In Vitro and In Vivo Diagnostic Substances',
    '2836': 'Biological Products, Except Diagnostic Substances',
    '3826': 'Laboratory Analytical Instruments',
    '3841': 'Surgical and Medical Instruments',
    '3842': 'Orthopedic, Prosthetic, and Surgical Supplies',
    '3843': 'Dental Equipment and Supplies',
    '3844': 'X-Ray Apparatus and Tubes',
    '3845': 'Electromedical and Electrotherapeutic Apparatus',
    '3851': 'Ophthalmic Goods',
    '8071': 'Medical Laboratories',
    '8731': 'Commercial Physical and Biological Research',
    '8732': 'Commercial Nonphysical Research',
    '8733': 'Noncommercial Research Organizations',
    '8734': 'Testing Laboratories',
    '8999': 'Services, Not Elsewhere Classified (including Medical Research)',
    '6324': 'Hospital and Medical Service Plans',
    '5122': 'Drugs, Proprietaries, and Druggists Sundries'
}

biotech_sic_codes = list(biotech_sic_dict.keys())

In [62]:
# Set up SEC API request headers
headers = {
    'User-Agent': 'sanspeursystems matt@sanspeursystems.com',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'data.sec.gov'
}

# Process each company
for idx, row in sick_df.iterrows():
    print(f"Processing {row['company']} ({idx + 1}/{len(sick_df)})")
    
    # Check company name for biotech keywords
    company_name_lower = row['company'].lower()
    matching_keywords = [kw for kw in biotech_keywords if kw in company_name_lower]
    if matching_keywords:
        sick_df.at[idx, 'is_biotech'] = True
        sick_df.at[idx, 'biotech_identification_method'] = f"name_match:{','.join(matching_keywords)}"
    
    # Get SEC data
    cik = row['cik']
    url = f'https://data.sec.gov/submissions/CIK{cik}.json'
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            
            # Store basic info
            sick_df.at[idx, 'ticker'] = data.get('tickers', [''])[0]
            sic_code = str(data.get('sic', '')).strip()
            sick_df.at[idx, 'sic'] = sic_code
            sick_df.at[idx, 'sic_description'] = data.get('sicDescription', '') or biotech_sic_dict.get(sic_code, '')
            
            # Check if SIC code matches biotech
            if sic_code in biotech_sic_codes:
                sick_df.at[idx, 'is_biotech'] = True
                current_method = sick_df.at[idx, 'biotech_identification_method']
                if current_method:
                    sick_df.at[idx, 'biotech_identification_method'] = f"{current_method};sic_{sic_code}"
                else:
                    sick_df.at[idx, 'biotech_identification_method'] = f"sic_{sic_code}"
            
            print(f"Company: {row['company']}")
            print(f"SIC: {sic_code} - {sick_df.at[idx, 'sic_description']}")
            print(f"Ticker: {sick_df.at[idx, 'ticker']}")
            print(f"Is Biotech: {sick_df.at[idx, 'is_biotech']}")
            print("---")
            
    except Exception as e:
        print(f"Error for {row['company']}: {str(e)}")
    
    time.sleep(0.1)  # Respect SEC rate limits

# Filter for biotech companies
btech_delisted_df = sick_df[sick_df['is_biotech']].copy()

# Display results
print("\nPotential Biotech Companies Found:")
print(btech_delisted_df[['date', 'company', 'ticker', 'sic', 'sic_description', 'biotech_identification_method']])

# Save results with detailed information
btech_delisted_df.to_csv('biotech_delistings_detailed.csv', index=False)

# Print summary statistics
print("\nSummary Statistics:")
print(f"Total companies processed: {len(sick_df)}")
print(f"Companies identified as biotech: {len(btech_delisted_df)}")
if len(sick_df) > 0:
    print(f"Percentage biotech: {(len(btech_delisted_df) / len(sick_df)) * 100:.1f}%")

# Print identification method breakdown
print("\nIdentification Method Breakdown:")
print(btech_delisted_df['biotech_identification_method'].value_counts())

# Print SIC code distribution for identified biotech companies
print("\nSIC Code Distribution for Identified Biotech Companies:")
sic_distribution = btech_delisted_df['sic'].value_counts()
for sic in sic_distribution.index:
    description = biotech_sic_dict.get(sic, 'Unknown')
    print(f"SIC {sic}: {description} - {sic_distribution[sic]} companies")

Processing Finch Therapeutics Group, Inc. (980/983)
Company: Finch Therapeutics Group, Inc.
SIC: 2836 - Biological Products, (No Diagnostic Substances)
Ticker: FNCH
Is Biotech: True
---
Processing TELLURIAN INC. /DE/ (983/983)
Company: TELLURIAN INC. /DE/
SIC: 1311 - Crude Petroleum & Natural Gas
Ticker: TELZ
Is Biotech: False
---
Processing Guardion Health Sciences, Inc. (979/983)
Company: Guardion Health Sciences, Inc.
SIC: 2834 - Pharmaceutical Preparations
Ticker: GHSI
Is Biotech: True
---
Processing Investment Managers Series Trust II (978/983)
Error for Investment Managers Series Trust II: list index out of range
Processing Renalytix plc (981/983)
Company: Renalytix plc
SIC: 8071 - Services-Medical Laboratories
Ticker: RNLXY
Is Biotech: True
---
Processing ORANGE (977/983)
Company: ORANGE
SIC: 4813 - Telephone Communications (No Radiotelephone)
Ticker: FNCTF
Is Biotech: False
---
Processing Generations Bancorp NY, Inc. (982/983)
Company: Generations Bancorp NY, Inc.
SIC: 6035 - S

In [63]:
print(len(btech_delisted_df))   
display(btech_delisted_df.head())

86


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
979,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836"
978,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834
980,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071
967,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826
962,2024-09-13,Calliditas Therapeutics AB,0001795579,25,,,,True,"name_match:therapeutics,therapeutic"


In [64]:
# print stocks with no ticker
print(len(btech_delisted_df[btech_delisted_df['ticker'] == '']))

27


In [65]:
# drop rows with no ticker
print(len(btech_delisted_df))
btech_delisted_df = btech_delisted_df[btech_delisted_df['ticker'] != '']
print(len(btech_delisted_df))

86
59


In [ ]:
# take a look at the data
display(btech_delisted_df)

In [67]:
from datetime import timedelta

In [68]:
#### ----
# add SEC delisting reasons data to df
#### ----
def add_sec_delisting_reasons(df):
    """
    Add SEC-based delisting reasons to existing biotech dataframe
    """
    # Add new columns for SEC information
    df['delisting_reason'] = ''
    df['sec_filings'] = ''
    
    # Set up SEC API headers
    headers = {
        'User-Agent': 'sanspeursystems matt@sanspeursystems.com',  
        'Accept-Encoding': 'gzip, deflate',
        'Host': 'data.sec.gov'
    }
    
    for idx, row in df.iterrows():
        print(f"Processing {row['ticker']} ({idx + 1}/{len(df)})")
        
        cik = str(row['cik']).zfill(10)  # Pad CIK with leading zeros
        delisting_date = pd.to_datetime(row['date'])
        start_date = delisting_date - timedelta(days=90)  # Look back 90 days
        
        try:
            # Get SEC filings
            url = f'https://data.sec.gov/submissions/CIK{cik}.json'
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                recent_filings = data.get('filings', {}).get('recent', {})
                
                if recent_filings:
                    forms = recent_filings.get('form', [])
                    dates = recent_filings.get('filingDate', [])
                    
                    relevant_filings = []
                    primary_reason = 'Unknown'
                    
                    # Check filings around delisting date
                    for form, file_date in zip(forms, dates):
                        file_date = pd.to_datetime(file_date)
                        
                        if start_date <= file_date <= delisting_date:
                            if form == '25':
                                primary_reason = 'Voluntary Delisting'
                                relevant_filings.append(f'Form 25 ({file_date.date()})')
                                
                            elif form in ['DEFM14A', 'PREM14A']:
                                primary_reason = 'Merger/Acquisition'
                                relevant_filings.append(f'Merger Proxy {form} ({file_date.date()})')
                                
                            elif form == '15':
                                if primary_reason == 'Unknown':
                                    primary_reason = 'Deregistration'
                                relevant_filings.append(f'Form 15 ({file_date.date()})')
                                
                            elif any(x in form for x in ['15-12B', '15-12G']):
                                primary_reason = 'Bankruptcy/Liquidation'
                                relevant_filings.append(f'{form} ({file_date.date()})')
                            
                            elif form == '8-K':
                                relevant_filings.append(f'8-K ({file_date.date()})')
                    
                    # Update dataframe
                    df.at[idx, 'delisting_reason'] = primary_reason
                    df.at[idx, 'sec_filings'] = '; '.join(relevant_filings)
            
        except Exception as e:
            print(f"Error processing {row['ticker']}: {str(e)}")
        
        time.sleep(0.1)  # Respect SEC rate limits
    
    return df

In [69]:
# execute - and add SEC delisting reasons data to df
btech_delisted_df = add_sec_delisting_reasons(btech_delisted_df)

Processing FNCH (980/59)
Processing GHSI (979/59)
Processing RNLXY (981/59)
Processing TBIO (968/59)
Processing MPSYF (955/59)
Processing AONC (944/59)
Processing VAXX (945/59)
Processing AMPE (934/59)
Processing LIANY (914/59)
Processing AMRX (875/59)
Processing PYRGF (877/59)
Processing NKGN (846/59)
Processing COO (851/59)
Processing AEON (844/59)
Processing BLUAF (819/59)
Processing ZURA (786/59)
Processing ZYME (759/59)
Processing CLDI (757/59)
Processing FOXO (750/59)
Processing FOXO (710/59)
Processing VANI (700/59)
Processing KLDO (684/59)
Processing SCPX (649/59)
Processing ANVS (617/59)
Processing ERNA (641/59)
Processing DNA (607/59)
Processing SRZN (605/59)
Processing ME (580/59)
Processing NSPR (575/59)
Processing ACB (577/59)
Processing NSPR (574/59)
Processing BIOX (578/59)
Processing OCX (555/59)
Processing CRMD (549/59)
Processing VTAK (560/59)
Processing CLOV (563/59)
Processing AMGN (541/59)
Processing CGC (538/59)
Processing AZN (513/59)
Processing CNMD (462/59)
Pro

In [79]:
print(len(btech_delisted_df))
display(btech_delisted_df.head())

59


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method,delisting_reason,sec_filings
979,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836",Voluntary Delisting,Form 25 (2024-10-31); 8-K (2024-10-21); 8-K (2...
978,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834,Voluntary Delisting,Form 25 (2024-10-11); 8-K (2024-10-03); 8-K (2...
980,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071,Voluntary Delisting,Form 25 (2024-10-07); 8-K (2024-10-01); 8-K (2...
967,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826,Voluntary Delisting,Form 25 (2024-09-20); 8-K (2024-09-10); 8-K (2...
954,2024-07-25,MorphoSys AG,0001340243,25,MPSYF,2834,Pharmaceutical Preparations,True,sic_2834,Voluntary Delisting,Form 25 (2024-07-25)


In [72]:
print(btech_delisted_df)

          date                                       company         cik form  \
979 2024-10-31                Finch Therapeutics Group, Inc.  0001733257   25   
978 2024-10-11                Guardion Health Sciences, Inc.  0001642375   25   
980 2024-10-07                                 Renalytix plc  0001811115   25   
967 2024-09-20                              Telesis Bio Inc.  0001850079   25   
954 2024-07-25                                  MorphoSys AG  0001340243   25   
943 2024-05-31               American Oncology Network, Inc.  0001839998   25   
944 2024-04-29                               Vaxxinity, Inc.  0001851657   25   
933 2024-04-04                   Ampio Pharmaceuticals, Inc.  0001411906   25   
913 2024-03-11                                       LianBio  0001831283   25   
874 2023-12-26                  Amneal Pharmaceuticals, Inc.  0001723128   25   
876 2023-11-06                       PyroGenesis Canada Inc.  0001743344   25   
845 2023-09-29              

reason filter time.

In [71]:
# create a breakdown grouping count of delisting reasons
print("\nDelisting Reason Breakdown:")
print(btech_delisted_df['delisting_reason'].value_counts())



Delisting Reason Breakdown:
delisting_reason
Voluntary Delisting       51
Merger/Acquisition         5
Unknown                    2
Bankruptcy/Liquidation     1
Name: count, dtype: int64


retrieving historical FUNDAMENTAL stock data
- board members for xboarding 
- free or paid? , yfinance

retrieving historical TECHNICAL stock $ data
- paid api? of paid

In [73]:
import yfinance as yf

In [77]:
# yfinance to pring as json all endpoints using stock PLTR
stock = yf.Ticker("PLTR")
print(stock.info)
# beautiful print
import json
print(json.dumps(stock.info, indent=4))


{'address1': '1200 17th Street', 'address2': 'Floor 15', 'city': 'Denver', 'state': 'CO', 'zip': '80202', 'country': 'United States', 'phone': '720 358 3679', 'website': 'https://www.palantir.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "Palantir Technologies Inc. builds and deploys software platforms for the intelligence community to assist in counterterrorism investigations and operations in the United States, the United Kingdom, and internationally. The company provides Palantir Gotham, a software platform which enables users to identify patterns hidden deep within datasets, ranging from signals intelligence sources to reports from confidential informants, as well as facilitates the handoff between analysts and operational users, helping operators plan and execute real-world responses to threats

In [74]:
def get_last_trading_data(df):
    """
    Retrieve last available trading data for each delisted biotech stock
    
    Args:
        df (pd.DataFrame): DataFrame containing delisted biotech stocks info
    """
    
    def get_stock_data(row):
        try:
            # Convert delisting date to datetime if it's a string
            end_date = pd.to_datetime(row['date'])
            # Set start date to 30 days before delisting to ensure we get data
            start_date = end_date - timedelta(days=30)
            
            # Create ticker object
            ticker = yf.Ticker(row['ticker'])
            
            # Get historical data
            hist = ticker.history(
                start=start_date.strftime('%Y-%m-%d'),
                end=end_date.strftime('%Y-%m-%d')
            )
            
            if hist.empty:
                return pd.Series({
                    'last_price': None,
                    'last_volume': None,
                    'last_date': None,
                    'market_cap': None,
                    'data_retrieval_status': 'No data available'
                })
            
            # Get the last row of data
            last_data = hist.iloc[-1]
            
            return pd.Series({
                'last_price': last_data['Close'],
                'last_volume': last_data['Volume'],
                'last_date': hist.index[-1],
                'market_cap': ticker.info.get('marketCap', None),
                'data_retrieval_status': 'Success'
            })
            
        except Exception as e:
            return pd.Series({
                'last_price': None,
                'last_volume': None,
                'last_date': None,
                'market_cap': None,
                'data_retrieval_status': f'Error: {str(e)}'
            })
    
    # Create results dataframe
    results_df = pd.DataFrame()
    
    # Process each stock with a delay to avoid rate limiting
    for idx, row in df.iterrows():
        print(f"Processing {row['ticker']} ({idx + 1}/{len(df)})")
        data = get_stock_data(row)
        
        # Create a single row dataframe with all information
        stock_data = pd.DataFrame({
            'ticker': [row['ticker']],
            'company': [row['company']],
            'delisting_date': [row['date']],
            'sic_code': [row['sic']],
            'sic_description': [row['sic_description']],
            'last_price': [data['last_price']],
            'last_volume': [data['last_volume']],
            'last_trading_date': [data['last_date']],
            'market_cap': [data['market_cap']],
            'status': [data['data_retrieval_status']]
        })
        
        results_df = pd.concat([results_df, stock_data], ignore_index=True)
        
        # Add delay to avoid rate limiting
        time.sleep(1)
    
    # Clean up the data
    results_df['last_price'] = pd.to_numeric(results_df['last_price'], errors='coerce')
    results_df['last_volume'] = pd.to_numeric(results_df['last_volume'], errors='coerce')
    results_df['market_cap'] = pd.to_numeric(results_df['market_cap'], errors='coerce')
    
    # Add some basic statistics
    results_df['market_cap_millions'] = results_df['market_cap'] / 1_000_000
    
    return results_df

def analyze_delisting_data(results_df):
    """
    Analyze the retrieved delisting data
    """
    summary = {
        'total_stocks': len(results_df),
        'successful_retrievals': len(results_df[results_df['status'] == 'Success']),
        'avg_last_price': results_df['last_price'].mean(),
        'avg_market_cap_millions': results_df['market_cap_millions'].mean(),
        'median_last_volume': results_df['last_volume'].median(),
        'sic_distribution': results_df['sic_code'].value_counts().to_dict()
    }
    
    return summary

# Usage:
# biotech_delisted_df = pd.read_csv('your_data.csv')  # Your input DataFrame
# results = get_last_trading_data(biotech_delisted_df)
# analysis = analyze_delisting_data(results)

In [75]:
# create btdl_enriched_df 
btdl_enriched_df = get_last_trading_data(btech_delisted_df)

print(len(btech_delisted_df))
print(len(btdl_enriched_df))
btdl_enriched_df.head()

Processing FNCH (980/59)
Processing GHSI (979/59)
Processing RNLXY (981/59)
Processing TBIO (968/59)
Processing MPSYF (955/59)
Processing AONC (944/59)
[04/Nov/2024 17:42:39] ERROR - $AONC: possibly delisted; no price data found  (1d 2024-05-01 -> 2024-05-31) (Yahoo error = "Data doesn't exist for startDate = 1714536000, endDate = 1717128000")


/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_81802/2480574920.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, stock_data], ignore_index=True)


Processing VAXX (945/59)
Processing AMPE (934/59)
Processing LIANY (914/59)
Processing AMRX (875/59)
Processing PYRGF (877/59)
Processing NKGN (846/59)
[04/Nov/2024 17:42:47] ERROR - $NKGN: possibly delisted; no price data found  (1d 2023-08-30 -> 2023-09-29) (Yahoo error = "Data doesn't exist for startDate = 1693368000, endDate = 1695960000")


/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_81802/2480574920.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, stock_data], ignore_index=True)


Processing COO (851/59)
Processing AEON (844/59)
[04/Nov/2024 17:42:50] ERROR - $AEON: possibly delisted; no price data found  (1d 2023-06-21 -> 2023-07-21) (Yahoo error = "Data doesn't exist for startDate = 1687320000, endDate = 1689912000")


...

/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_81802/2480574920.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, stock_data], ignore_index=True)


Processing PGEN (253/59)
Processing NMTRQ (207/59)
Processing QGEN (198/59)
Processing TXMD (185/59)
Processing LAB (144/59)
Processing ALT (120/59)
[04/Nov/2024 17:43:41] ERROR - $ALT: possibly delisted; no price data found  (1d 2017-04-05 -> 2017-05-05) (Yahoo error = "Data doesn't exist for startDate = 1491364800, endDate = 1493956800")


/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_81802/2480574920.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, stock_data], ignore_index=True)


Processing SVRA (119/59)
[04/Nov/2024 17:43:43] ERROR - $SVRA: possibly delisted; no price data found  (1d 2017-03-28 -> 2017-04-27) (Yahoo error = "Data doesn't exist for startDate = 1490673600, endDate = 1493265600")
Processing PBYI (90/59)
Processing NWBO (75/59)
Processing ONVO (62/59)
Processing OPK (48/59)
Processing TRUP (32/59)
Processing MASI (17/59)
59
59


,ticker,company,delisting_date,sic_code,sic_description,last_price,last_volume,last_trading_date,market_cap,status,market_cap_millions
0,FNCH,"Finch Therapeutics Group, Inc.",2024-10-31,2836,"Biological Products, (No Diagnostic Substances)",11.900000,400.0,2024-10-30 00:00:00-04:00,1.918883e+07,Success,19.188832
1,GHSI,"Guardion Health Sciences, Inc.",2024-10-11,2834,Pharmaceutical Preparations,3.260000,36861.0,2024-10-10 00:00:00-04:00,4.346871e+06,Success,4.346871
2,RNLXY,Renalytix plc,2024-10-07,8071,Services-Medical Laboratories,0.196000,284300.0,2024-10-04 00:00:00-04:00,1.959614e+07,Success,19.596142
3,TBIO,Telesis Bio Inc.,2024-09-20,3826,Laboratory Analytical Instruments,2.930000,53735400.0,2024-09-19 00:00:00-04:00,2.570777e+06,Success,2.570777
4,MPSYF,MorphoSys AG,2024-07-25,2834,Pharmaceutical Preparations,73.269997,0.0,2024-07-24 00:00:00-04:00,2.759546e+09,Success,2759.545856


In [78]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import time
import json

def get_stock_fundamentals(df):
    """
    Retrieve fundamental data including company officers and sector information
    """
    
    def get_officer_details(officers):
        """Helper function to format officer information"""
        if not officers:
            return None
        
        officer_list = []
        for officer in officers:
            officer_info = {
                'name': officer.get('name'),
                'title': officer.get('title'),
                'age': officer.get('age'),
                'yearBorn': officer.get('yearBorn'),
                'totalPay': officer.get('totalPay')
            }
            officer_list.append(officer_info)
        
        return json.dumps(officer_list)  # Store as JSON string

    def get_fundamentals(row):
        try:
            # Create ticker object
            ticker = yf.Ticker(row['ticker'])
            
            # Get last price data
            end_date = pd.to_datetime(row['date'])
            start_date = end_date - timedelta(days=30)
            
            hist = ticker.history(
                start=start_date.strftime('%Y-%m-%d'),
                end=end_date.strftime('%Y-%m-%d')
            )
            
            last_price = hist['Close'].iloc[-1] if not hist.empty else None
            
            # Get fundamental data
            info = ticker.info
            
            # Basic company information
            results = {
                'last_price': last_price,
                'sector': info.get('sector'),
                'industry': info.get('industry'),
                'company_officers': get_officer_details(info.get('companyOfficers')),
                
                # Financial metrics
                'market_cap': info.get('marketCap'),
                'enterprise_value': info.get('enterpriseValue'),
                'total_cash': info.get('totalCash'),
                'total_debt': info.get('totalDebt'),
                'total_revenue': info.get('totalRevenue'),
                'gross_profits': info.get('grossProfits'),
                'free_cashflow': info.get('freeCashflow'),
                'research_development': info.get('researchAndDevelopment'),
                
                # Additional metrics
                'number_of_employees': info.get('fullTimeEmployees'),
                'shares_outstanding': info.get('sharesOutstanding'),
                'float_shares': info.get('floatShares'),
                'beta': info.get('beta'),
                
                # Status
                'data_retrieval_status': 'Success'
            }
            
            return pd.Series(results)
            
        except Exception as e:
            # Create empty results
            results = {
                'last_price': None,
                'sector': None,
                'industry': None,
                'company_officers': None,
                'market_cap': None,
                'enterprise_value': None,
                'total_cash': None,
                'total_debt': None,
                'total_revenue': None,
                'gross_profits': None,
                'free_cashflow': None,
                'research_development': None,
                'number_of_employees': None,
                'shares_outstanding': None,
                'float_shares': None,
                'beta': None,
                'data_retrieval_status': f'Error: {str(e)}'
            }
            
            return pd.Series(results)
    
    # Create results dataframe
    results_df = pd.DataFrame()
    
    # Process each stock
    total_stocks = len(df)
    for idx, row in df.iterrows():
        print(f"Processing {row['ticker']} ({idx + 1}/{total_stocks})")
        
        # Get fundamental data
        fund_data = get_fundamentals(row)
        
        # Create a single row dataframe
        stock_data = pd.DataFrame({
            'ticker': [row['ticker']],
            'company': [row['company']],
            'delisting_date': [row['date']],
            'sic_code': [row['sic']],
            'sic_description': [row['sic_description']],
            **{k: [v] for k, v in fund_data.items()}
        })
        
        results_df = pd.concat([results_df, stock_data], ignore_index=True)
        
        # Add delay to avoid rate limiting
        time.sleep(1.5)
    
    # Convert numeric columns
    numeric_columns = [
        'last_price', 'market_cap', 'enterprise_value', 'total_cash', 
        'total_debt', 'total_revenue', 'gross_profits', 'free_cashflow',
        'research_development', 'number_of_employees', 'shares_outstanding',
        'float_shares', 'beta'
    ]
    
    for col in numeric_columns:
        results_df[col] = pd.to_numeric(results_df[col], errors='coerce')
    
    # Add derived metrics
    results_df['net_cash'] = results_df['total_cash'] - results_df['total_debt']
    results_df['cash_to_debt_ratio'] = results_df['total_cash'] / results_df['total_debt']
    results_df['rd_to_revenue_ratio'] = results_df['research_development'] / results_df['total_revenue']
    
    return results_df

def analyze_officers(results_df):
    """
    Analyze the company officers data
    """
    officer_analysis = []
    
    for _, row in results_df.iterrows():
        if row['company_officers']:
            try:
                officers = json.loads(row['company_officers'])
                officer_analysis.append({
                    'ticker': row['ticker'],
                    'company': row['company'],
                    'number_of_officers': len(officers),
                    'officers': officers
                })
            except:
                continue
    
    return pd.DataFrame(officer_analysis)

def summarize_results(results_df):
    """
    Create a summary of the results
    """
    summary = {
        'total_stocks': len(results_df),
        'successful_retrievals': len(results_df[results_df['data_retrieval_status'] == 'Success']),
        'sectors_distribution': results_df['sector'].value_counts().to_dict(),
        'avg_metrics': {
            'market_cap': results_df['market_cap'].mean(),
            'total_cash': results_df['total_cash'].mean(),
            'total_debt': results_df['total_debt'].mean(),
            'rd_expense': results_df['research_development'].mean()
        }
    }
    
    # Get officer statistics
    officer_df = analyze_officers(results_df)
    if not officer_df.empty:
        summary['officer_stats'] = {
            'avg_officers_per_company': officer_df['number_of_officers'].mean(),
            'max_officers': officer_df['number_of_officers'].max(),
            'companies_with_officer_data': len(officer_df)
        }
    
    return summary

# Example usage:
# results = get_stock_fundamentals(biotech_delisted_df)
# summary = summarize_results(results)
# officer_details = analyze_officers(results)

In [81]:
# Get the full data
bt_officers_df = get_stock_fundamentals(btech_delisted_df)

print(len(btech_delisted_df))
display(bt_officers_df.head())

# # Get officer-specific analysis
# officer_analysis = analyze_officers(results)

# # Get overall summary
# summary = summarize_results(results)

# # View officer details for a specific company
# company_officers = results.loc[results['ticker'] == 'SPECIFIC_TICKER', 'company_officers'].iloc[0]
# if company_officers:
#     print(json.loads(company_officers))

Processing FNCH (980/59)
Processing GHSI (979/59)
Processing RNLXY (981/59)
Processing TBIO (968/59)
Processing MPSYF (955/59)


/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_81802/4102864954.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, stock_data], ignore_index=True)


Processing AONC (944/59)
[04/Nov/2024 17:54:22] ERROR - $AONC: possibly delisted; no price data found  (1d 2024-05-01 -> 2024-05-31) (Yahoo error = "Data doesn't exist for startDate = 1714536000, endDate = 1717128000")
Processing VAXX (945/59)
Processing AMPE (934/59)
Processing LIANY (914/59)
Processing AMRX (875/59)
Processing PYRGF (877/59)
Processing NKGN (846/59)
[04/Nov/2024 17:54:34] ERROR - $NKGN: possibly delisted; no price data found  (1d 2023-08-30 -> 2023-09-29) (Yahoo error = "Data doesn't exist for startDate = 1693368000, endDate = 1695960000")


/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_81802/4102864954.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, stock_data], ignore_index=True)


Processing COO (851/59)
Processing AEON (844/59)
[04/Nov/2024 17:54:38] ERROR - $AEON: possibly delisted; no price data found  (1d 2023-06-21 -> 2023-07-21) (Yahoo error = "Data doesn't exist for startDate = 1687320000, endDate = 1689912000")


...

/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_81802/4102864954.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, stock_data], ignore_index=True)


Processing QGEN (198/59)
Processing TXMD (185/59)
Processing LAB (144/59)
Processing ALT (120/59)
[04/Nov/2024 17:55:49] ERROR - $ALT: possibly delisted; no price data found  (1d 2017-04-05 -> 2017-05-05) (Yahoo error = "Data doesn't exist for startDate = 1491364800, endDate = 1493956800")


/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_81802/4102864954.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, stock_data], ignore_index=True)


Processing SVRA (119/59)
[04/Nov/2024 17:55:51] ERROR - $SVRA: possibly delisted; no price data found  (1d 2017-03-28 -> 2017-04-27) (Yahoo error = "Data doesn't exist for startDate = 1490673600, endDate = 1493265600")
Processing PBYI (90/59)
Processing NWBO (75/59)
Processing ONVO (62/59)
Processing OPK (48/59)
Processing TRUP (32/59)
Processing MASI (17/59)
59


,ticker,company,delisting_date,sic_code,sic_description,last_price,sector,industry,company_officers,market_cap,...,free_cashflow,research_development,number_of_employees,shares_outstanding,float_shares,beta,data_retrieval_status,net_cash,cash_to_debt_ratio,rd_to_revenue_ratio
0,FNCH,"Finch Therapeutics Group, Inc.",2024-10-31,2836,"Biological Products, (No Diagnostic Substances)",11.900000,Healthcare,Biotechnology,"[{""name"": ""Mr. Matthew P. Blischak J.D."", ""tit...",1.918883e+07,...,-15963625.0,NaN,1.0,1605760.0,896016.0,1.329,Success,-1.295400e+07,0.553141,NaN
1,GHSI,"Guardion Health Sciences, Inc.",2024-10-11,2834,Pharmaceutical Preparations,3.260000,Healthcare,Drug Manufacturers - Specialty & Generic,"[{""name"": ""Mr. Robert Neal Weingarten"", ""title...",4.346871e+06,...,1377253.0,NaN,9.0,1349960.0,1160582.0,0.708,Success,NaN,NaN,NaN
2,RNLXY,Renalytix plc,2024-10-07,8071,Services-Medical Laboratories,0.196000,Healthcare,Health Information Services,"[{""name"": ""Mr. James R. McCullough M.B.A."", ""t...",1.959614e+07,...,-19473124.0,NaN,NaN,94966600.0,139219802.0,2.103,Success,-3.855000e+06,0.548330,NaN
3,TBIO,Telesis Bio Inc.,2024-09-20,3826,Laboratory Analytical Instruments,2.930000,Healthcare,Medical Devices,"[{""name"": ""Mr. Eric Esser"", ""title"": ""Preside...",2.570777e+06,...,-9338625.0,NaN,101.0,1772950.0,612110.0,1.758,Success,-2.533900e+07,0.287289,NaN
4,MPSYF,MorphoSys AG,2024-07-25,2834,Pharmaceutical Preparations,73.269997,None,None,"[{""name"": ""Ms. Charlotte Lohmann"", ""title"": ""...",2.759546e+09,...,-29450064.0,NaN,524.0,37662700.0,NaN,NaN,Success,-1.233860e+09,0.293391,NaN


In [84]:
print(len(btech_delisted_df))
print(len(bt_officers_df))

59
59


In [93]:
print(len(btech_delisted_df) == len(bt_officers_df))
print(len(btech_delisted_df), len(bt_officers_df))

print('\nDATA MINING OUTPUT\n')

print('btech OFFICERS')
display(bt_officers_df.head())
display(bt_officers_df.columns)
print('btech SICK data mining output')
display(btech_delisted_df.head())
display(btech_delisted_df.columns)


True
59 59

DATA MINING OUTPUT

btech OFFICERS


,ticker,company,delisting_date,sic_code,sic_description,last_price,sector,industry,company_officers,market_cap,...,free_cashflow,research_development,number_of_employees,shares_outstanding,float_shares,beta,data_retrieval_status,net_cash,cash_to_debt_ratio,rd_to_revenue_ratio
0,FNCH,"Finch Therapeutics Group, Inc.",2024-10-31,2836,"Biological Products, (No Diagnostic Substances)",11.900000,Healthcare,Biotechnology,"[{""name"": ""Mr. Matthew P. Blischak J.D."", ""tit...",1.918883e+07,...,-15963625.0,NaN,1.0,1605760.0,896016.0,1.329,Success,-1.295400e+07,0.553141,NaN
1,GHSI,"Guardion Health Sciences, Inc.",2024-10-11,2834,Pharmaceutical Preparations,3.260000,Healthcare,Drug Manufacturers - Specialty & Generic,"[{""name"": ""Mr. Robert Neal Weingarten"", ""title...",4.346871e+06,...,1377253.0,NaN,9.0,1349960.0,1160582.0,0.708,Success,NaN,NaN,NaN
2,RNLXY,Renalytix plc,2024-10-07,8071,Services-Medical Laboratories,0.196000,Healthcare,Health Information Services,"[{""name"": ""Mr. James R. McCullough M.B.A."", ""t...",1.959614e+07,...,-19473124.0,NaN,NaN,94966600.0,139219802.0,2.103,Success,-3.855000e+06,0.548330,NaN
3,TBIO,Telesis Bio Inc.,2024-09-20,3826,Laboratory Analytical Instruments,2.930000,Healthcare,Medical Devices,"[{""name"": ""Mr. Eric Esser"", ""title"": ""Preside...",2.570777e+06,...,-9338625.0,NaN,101.0,1772950.0,612110.0,1.758,Success,-2.533900e+07,0.287289,NaN
4,MPSYF,MorphoSys AG,2024-07-25,2834,Pharmaceutical Preparations,73.269997,None,None,"[{""name"": ""Ms. Charlotte Lohmann"", ""title"": ""...",2.759546e+09,...,-29450064.0,NaN,524.0,37662700.0,NaN,NaN,Success,-1.233860e+09,0.293391,NaN


Index(['ticker', 'company', 'delisting_date', 'sic_code', 'sic_description',
       'last_price', 'sector', 'industry', 'company_officers', 'market_cap',
       'enterprise_value', 'total_cash', 'total_debt', 'total_revenue',
       'gross_profits', 'free_cashflow', 'research_development',
       'number_of_employees', 'shares_outstanding', 'float_shares', 'beta',
       'data_retrieval_status', 'net_cash', 'cash_to_debt_ratio',
       'rd_to_revenue_ratio'],
      dtype='object')

btech SICK data mining output


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method,delisting_reason,sec_filings
979,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836",Voluntary Delisting,Form 25 (2024-10-31); 8-K (2024-10-21); 8-K (2...
978,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834,Voluntary Delisting,Form 25 (2024-10-11); 8-K (2024-10-03); 8-K (2...
980,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071,Voluntary Delisting,Form 25 (2024-10-07); 8-K (2024-10-01); 8-K (2...
967,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826,Voluntary Delisting,Form 25 (2024-09-20); 8-K (2024-09-10); 8-K (2...
954,2024-07-25,MorphoSys AG,0001340243,25,MPSYF,2834,Pharmaceutical Preparations,True,sic_2834,Voluntary Delisting,Form 25 (2024-07-25)


Index(['date', 'company', 'cik', 'form', 'ticker', 'sic', 'sic_description',
       'is_biotech', 'biotech_identification_method', 'delisting_reason',
       'sec_filings'],
      dtype='object')

figure out in what way to effectively to combine this data

make your next dataframe historical price and chart, technicals 
